# Step 1: Imports + setup

In [2]:
import pandas as pd
import json
from pathlib import Path

# Step 2: Load BLS data (CSV-like pipe-delimited file)

In [3]:
bls_path = Path("../data/raw/bls/pr.data.0.Current")
bls_df = pd.read_csv(
    bls_path,
    sep="\t",
    dtype = {"series_id":str,"period": str}
)

bls_df.head()

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


# Step 3: Basic cleaning

In [27]:
# Strip whitespaces from column names
bls_df.columns = bls_df.columns.str.strip()

# Strip whitespace from string columns
bls_df["year"] = bls_df["year"].astype(int)
bls_df["value"] = bls_df["value"].astype(int)

# Strip whitespace from string columns
bls_df["series_id"] = bls_df["series_id"].astype(str).str.strip()
bls_df["period"] = bls_df["period"].astype(str).str.strip()

bls_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 37521 entries, 0 to 37520
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   series_id       37521 non-null  str  
 1   year            37521 non-null  int64
 2   period          37521 non-null  str  
 3   value           37521 non-null  int64
 4   footnote_codes  97 non-null     str  
dtypes: int64(2), str(3)
memory usage: 1.4 MB


# Step 4: Load population JSON

In [28]:
api_dir = Path("../data/raw/api")
population_file = sorted(api_dir.glob("population_*.json"))[-1]

with open(population_file, "r") as f:
    population_json =json.load(f)

population_df = pd.DataFrame(population_json["data"])
population_df.head()

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0


# Step 5: Clean population dataframe

In [29]:
population_df.columns = population_df.columns.str.strip()

population_df["Year"] = population_df["Year"].astype(int)
population_df["Population"] = pd.to_numeric(
    population_df["Population"], errors="coerce"
)

population_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Nation ID   10 non-null     str    
 1   Nation      10 non-null     str    
 2   Year        10 non-null     int64  
 3   Population  10 non-null     float64
dtypes: float64(1), int64(1), str(2)
memory usage: 452.0 bytes


# Step 6: Filter population data to required years

In [30]:
# Filter population data for years 2013 - 2018 inclusive
pop_2013_2018 = population_df[
    (population_df["Year"]>=2013) &
    (population_df["Year"]<=2018)
]

pop_2013_2018

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0


# Step 7: Compute mean and standard deviation

In [31]:
population_mean = pop_2013_2018["Population"].mean()
population_std = pop_2013_2018["Population"].std()
print(f"Mean US Population (2013 - 2018): {population_mean:,.0f}")
print(f"Std deviation of US population (2013 - 2018): {population_std:,.0f}")

Mean US Population (2013 - 2018): 322,069,808
Std deviation of US population (2013 - 2018): 4,158,441


# Step 9 – Aggregate Best Years for each series

In [36]:
# Group by series_id and year, sum the values
yearly_sums = bls_df.groupby(['series_id', 'year'])['value'].sum().reset_index()

# For each series_id, find the row with the maximum summed value
best_years = yearly_sums.loc[yearly_sums.groupby('series_id')['value'].idxmax()]

# Display the report
best_years.head()

,series_id,year,value
27,PRS30006011,2022,19
58,PRS30006012,2022,15
65,PRS30006013,1998,703
108,PRS30006021,2010,16
139,PRS30006022,2010,11


# Step 10 – Final Join Report

10.1 — Filter BLS

In [38]:
target_bls = bls_df[
    (bls_df["series_id"] == 'PRS30006032') &
    (bls_df["period"]=='Q01')
]
target_bls.head()

,series_id,year,period,value,footnote_codes
1071,PRS30006032,1995,Q01,0,NaN
1076,PRS30006032,1996,Q01,-4,NaN
1081,PRS30006032,1997,Q01,2,NaN
1086,PRS30006032,1998,Q01,0,NaN
1091,PRS30006032,1999,Q01,-4,NaN


10.2 — Prepare population dataframe

In [39]:
population_for_join = population_df[[
    "Year", "Population"]].rename(columns={"Year": "year"})

10.3 — Join on year

In [40]:
final_report = target_bls.merge(
    population_for_join,
    on = 'year',
    how = 'left'
)
final_report

,series_id,year,period,value,footnote_codes,Population
0,PRS30006032,1995,Q01,0,NaN,NaN
1,PRS30006032,1996,Q01,-4,NaN,NaN
2,PRS30006032,1997,Q01,2,NaN,NaN
3,PRS30006032,1998,Q01,0,NaN,NaN
4,PRS30006032,1999,Q01,-4,NaN,NaN
5,PRS30006032,2000,Q01,0,NaN,NaN
6,PRS30006032,2001,Q01,-6,NaN,NaN
7,PRS30006032,2002,Q01,-6,NaN,NaN
8,PRS30006032,2003,Q01,-5,NaN,NaN
9,PRS30006032,2004,Q01,2,NaN,NaN
